## This Notebook handels the Dualmyo Database 

In this notebook the paper EMG-based online classification of gestures with recurrent neural networks will be copied

8 Gestures, 16 Channel-EMG placed on the the wirst and 110 repitions, 5 recordings sessions, 200Hz, 95 synthetic sequences

Each sequence has a lenth of about 50sec (about 10000 timestemps)

https://zenodo.org/record/1320922#.Y9DlgtLMJhE

paper: EMG-based online classification of gestures with recurrent neural networks

In [1]:
import pickle
import gzip
import os
import numpy as np
import random

In [2]:
path = os.path.abspath("/home/finn/Dokumente/EMG_data/UC2018 DualMyo Hand Gesture Dataset/dualmyo_dataset.pkl")

In [3]:
with open(path, 'rb') as f:
    data = pickle.load(f)

In [52]:
# extract independant and dependant data 
# independant data like samples, length, EMG-channels
x, y = data[0], data[1]
x = np.concatenate([sample.reshape(((1,) + sample.shape)) for sample in x])
y = np.array(y)
x.shape

(881, 400, 20)

In [99]:
np.unique(y, return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6, 7]),
 array([110, 110, 110, 110, 111, 110, 110, 110]))

In [98]:
np.where(y == 0)

(array([  5,  14,  16,  30,  33,  41,  53,  63,  64,  72,  82,  90, 103,
        105, 116, 120, 134, 138, 149, 155, 163, 171, 176, 190, 193, 206,
        215, 219, 231, 237, 244, 249, 258, 269, 278, 287, 293, 296, 306,
        318, 324, 335, 341, 351, 358, 360, 369, 378, 384, 397, 403, 414,
        418, 427, 434, 446, 450, 456, 467, 479, 481, 493, 503, 510, 515,
        524, 530, 539, 546, 559, 564, 572, 582, 586, 595, 607, 608, 620,
        626, 639, 646, 652, 656, 664, 672, 680, 692, 700, 711, 716, 724,
        732, 737, 752, 758, 768, 769, 780, 792, 797, 806, 816, 821, 825,
        830, 843, 848, 859, 869, 870]),)

In [133]:
# generate sequences consisting of n = 8 gestures
# set random seeds 
random.seed(42)
np.random.seed(42)

n = 8
length = len(x) // n
indx_shuf = np.random.permutation(len(y))
indx_shuf = indx_shuf[:-(len(indx_shuf) % n)] # remove one gesture as it is not usable
indx_shuf = indx_shuf.reshape(110, 8)
# implement rest gesture between gestures and do transiton between gestures
indx_shuf

array([[350, 254, 490, 688, 393,  30, 572, 791],
       [433, 736, 651, 679, 294, 833, 737, 468],
       [314, 428, 368, 678, 382,  44, 338,  23],
       [344, 795, 292, 804, 656, 300, 521,  66],
       [332, 192, 432, 291,  39, 580, 365,  72],
       [139,  76, 874, 494, 711, 250, 777, 404],
       [ 70, 613, 299, 828, 530, 213, 826, 355],
       [597, 537, 280, 110, 331, 714, 578, 531],
       [465, 420,  67, 323, 196, 810, 305, 870],
       [367, 429, 603, 493, 811, 309,  86, 684],
       [839, 552, 695, 136, 275, 120, 861, 485],
       [685,  96, 866, 629, 311, 231, 876, 265],
       [342, 717, 319, 744, 388, 730, 198,  63],
       [570, 840, 137, 697,  65, 215, 513, 832],
       [ 78, 705, 174, 591, 728, 611, 456, 412],
       [168, 477, 210, 812, 582,  49, 644, 424],
       [411, 616, 875, 821, 709,  33,  31, 687],
       [665, 499, 655, 109,  77, 793,  82, 306],
       [576, 208, 635, 784, 363,   5,  54, 227],
       [442, 878, 583,  97, 816, 843, 758, 541],
       [ 25,  84,  1

In [147]:
# break samples into training, validation and test set 
x_train = x[indx_shuf[:int(0.6 * len(indx_shuf))]]
x_val = x[indx_shuf[int(0.6 * len(indx_shuf)):int(0.8 * len(indx_shuf))]]
x_test = x[indx_shuf[int(0.8 * len(indx_shuf)):]]

x_train

array([[[[ 0.00000000e+00,  1.52888268e+09,  2.00000000e+00, ...,
          -1.00000000e+01, -1.60000000e+01,  9.70000000e+01],
         [ 0.00000000e+00,  1.52888268e+09,  2.00000000e+00, ...,
           6.00000000e+00, -3.60000000e+01, -3.10000000e+01],
         [ 1.00000000e+00,  1.52888268e+09, -8.00000000e+00, ...,
          -1.90000000e+01,  1.00000000e+01,  3.00000000e+01],
         ...,
         [ 1.00000000e+00,  1.52888268e+09,  1.00000000e+01, ...,
           5.00000000e+00, -7.00000000e+00, -5.70000000e+01],
         [ 2.00000000e+00,  1.52888268e+09, -9.00000000e+00, ...,
           3.00000000e+00,  3.00000000e+00,  1.90000000e+01],
         [ 2.00000000e+00,  1.52888268e+09, -6.00000000e+00, ...,
          -2.70000000e+01, -1.50000000e+01, -1.90000000e+01]],

        [[ 1.00000000e+00,  1.52882443e+09, -1.50000000e+01, ...,
           4.00000000e+00,  2.90000000e+01,  0.00000000e+00],
         [ 1.00000000e+00,  1.52882443e+09,  2.30000000e+01, ...,
          -2.50000000e